### 머신러닝 문제
1. bike.csv 파일을 불러오세요.

* datetime : 기록 일시(1시간 간격)
* season : 계절(봄 : 1, 여름 : 2, 가을 : 3, 겨울 : 4)
* holiday : 공휴일 여부(0 : 공휴일 아님, 1 : 공휴일)
* workingday : 근무일 여부(0 : 근무일 아님, 1 : 근무일)
* weather : 날씨(1 : 맑음, 2 : 옅은 안개, 3 : 약한 눈 or 약한 번개 or 흐림, 4 : 폭우 or 짙은 안개)
* temp : 실제 온도
* atemp : 체감 온도
* humidity : 상대 습도
* windspeed : 풍속
* casual : 등록되지 않은 사용자(비회원 수)
* registered : 등록된 사용자(회원) 수
* count : 자전거 대여 수량

In [ ]:
import pandas as pd
bike=pd.read_csv('bike.csv')

2. 자전거 대여 수요 예측을 해주는 프로그램을 만들어보세요. 다양한 모델을 활용해서 가장 높은 정확도를 이끌어보세요.  
(count열이 자전거의 수요값입니다. 다양한 특성을 활용해서 자전거의 수요를 예측해보세요.)

수치값에 대한 예측이니 분류모델이 아닌 회귀모델

배운 회귀모델은 2가지
- 선형회귀
- 랜덤포레스트회귀
- K최근접이웃회귀모델
- 에서 선형회귀랑 랜덤포레스트회귀 모델 사용

----score가 가장 높게 나온 모델----

In [ ]:
import pandas as pd
bike=pd.read_csv('bike.csv')
bike['datetime']=pd.to_datetime(bike['datetime'])
bike["year"] = bike["datetime"].dt.year
bike["month"] = bike["datetime"].dt.month
bike["day"] = bike["datetime"].dt.day
bike["hour"] = bike["datetime"].dt.hour
target=bike['count']
data=bike.drop(['datetime','casual','registered','count'],axis=1)

from sklearn.model_selection import train_test_split
train_data, test_data, train_target, test_target=train_test_split(data,target,test_size=0.1,random_state=10)

from sklearn.ensemble import RandomForestRegressor
rf=RandomForestRegressor()
rf.fit(train_data,train_target)
score=rf.score(test_data,test_target)

from sklearn.model_selection import GridSearchCV
dic={'max_depth' : range(5,16)}
gs=GridSearchCV(estimator=rf, param_grid=dic, cv=5)
gs.fit(train_data, train_target)
best_para=gs.best_params_
gs_result=gs.cv_results_
best_est=gs.best_estimator_

print(score)
print(best_para)
print(best_est)
best_est.score(test_data, test_target)

0.9460531255966756
{'max_depth': 15}
RandomForestRegressor(max_depth=15)


0.9438735137751703

----접근했던 풀이----

데이터 전처리

In [ ]:
bike.info()
# nan값은 없음
# Dtype이 object인 datatime을 숫자로 변환필요

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   datetime    10886 non-null  object 
 1   season      10886 non-null  int64  
 2   holiday     10886 non-null  int64  
 3   workingday  10886 non-null  int64  
 4   weather     10886 non-null  int64  
 5   temp        10886 non-null  float64
 6   atemp       10886 non-null  float64
 7   humidity    10886 non-null  int64  
 8   windspeed   10886 non-null  float64
 9   casual      10886 non-null  int64  
 10  registered  10886 non-null  int64  
 11  count       10886 non-null  int64  
dtypes: float64(3), int64(8), object(1)
memory usage: 1020.7+ KB


In [ ]:
bike['datetime']=pd.to_datetime(bike['datetime'])

bike["year"] = bike["datetime"].dt.year
bike["month"] = bike["datetime"].dt.month
bike["day"] = bike["datetime"].dt.day
bike["hour"] = bike["datetime"].dt.hour

target=bike['count']
data=bike.drop(['datetime','casual','registered','count'],axis=1)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   season      10886 non-null  int64  
 1   holiday     10886 non-null  int64  
 2   workingday  10886 non-null  int64  
 3   weather     10886 non-null  int64  
 4   temp        10886 non-null  float64
 5   atemp       10886 non-null  float64
 6   humidity    10886 non-null  int64  
 7   windspeed   10886 non-null  float64
 8   year        10886 non-null  int64  
 9   month       10886 non-null  int64  
 10  day         10886 non-null  int64  
 11  hour        10886 non-null  int64  
dtypes: float64(3), int64(9)
memory usage: 1020.7 KB


random_state=10 고정

선형 회귀 모델

In [ ]:
# 데이터 전처리
import pandas as pd
bike=pd.read_csv('bike.csv')
bike['datetime']=pd.to_datetime(bike['datetime'])
bike["year"] = bike["datetime"].dt.year
bike["month"] = bike["datetime"].dt.month
bike["day"] = bike["datetime"].dt.day
bike["hour"] = bike["datetime"].dt.hour
target=bike['count']
data=bike.drop(['datetime','casual','registered','count'],axis=1)

# 데이터 분할
from sklearn.model_selection import train_test_split
train_data, test_data, train_target, test_target = train_test_split(data, target, random_state=10)

# 데이터 스케일링(StandardScaler)
from sklearn.preprocessing import StandardScaler
ss=StandardScaler()
train_scaled=ss.fit_transform(train_data)
test_scaled=ss.transform(test_data)

# 선형 회귀모델
from sklearn.linear_model import LinearRegression
lr=LinearRegression()
lr.fit(train_scaled, train_target)
score=lr.score(test_scaled,test_target)

# 교차검증
from sklearn.model_selection import cross_val_score
cross_val=cross_val_score(lr,test_scaled, test_target,cv=5)

print('단독점수:',score)
print('교차검증:', cross_val)
print('교차검증 평균:',cross_val.mean())

단독점수: 0.38548214323578955
교차검증: [0.38309563 0.38386109 0.40436611 0.38789065 0.37568423]
교차검증 평균: 0.3869795433948723


In [ ]:
import pandas as pd
bike=pd.read_csv('bike.csv')
bike['datetime']=pd.to_datetime(bike['datetime'])
bike["year"] = bike["datetime"].dt.year
bike["month"] = bike["datetime"].dt.month
bike["day"] = bike["datetime"].dt.day
bike["hour"] = bike["datetime"].dt.hour
target=bike['count']
data=bike.drop(['datetime','casual','registered','count'],axis=1)

from sklearn.model_selection import train_test_split
train_data, test_data, train_target, test_target = train_test_split(data, target,random_state=10)

# 데이터 스케일링(MinMaxScaler)
from sklearn.preprocessing import MinMaxScaler
ss=MinMaxScaler()
train_scaled=ss.fit_transform(train_data)
test_scaled=ss.transform(test_data)

from sklearn.linear_model import LinearRegression
lr=LinearRegression()
lr.fit(train_scaled, train_target)
score=lr.score(test_scaled,test_target)

from sklearn.model_selection import cross_val_score
cross_val=cross_val_score(lr,test_scaled, test_target,cv=5)

print('----------------------------\n MinMaxScaler 결과')
print('단독점수:',score)
print('교차검증:', cross_val)
print('교차검증 평균:',cross_val.mean())

# 데이터 스케일링(RobustScaler)
from sklearn.preprocessing import RobustScaler
ss=RobustScaler()
train_scaled=ss.fit_transform(train_data)
test_scaled=ss.transform(test_data)

from sklearn.linear_model import LinearRegression
lr=LinearRegression()
lr.fit(train_scaled, train_target)
score=lr.score(test_scaled,test_target)

from sklearn.model_selection import cross_val_score
cross_val=cross_val_score(lr,test_scaled, test_target,cv=5)
print('----------------------------\n RobustScaler 결과')
print('단독점수:',score)
print('교차검증:', cross_val)
print('교차검증 평균:',cross_val.mean())

----------------------------
 MinMaxScaler 결과
단독점수: 0.38548214323578955
교차검증: [0.38309563 0.38386109 0.40436611 0.38789065 0.37568423]
교차검증 평균: 0.3869795433948722
----------------------------
 RobustScaler 결과
단독점수: 0.38548214323578955
교차검증: [0.38309563 0.38386109 0.40436611 0.38789065 0.37568423]
교차검증 평균: 0.38697954339487234


데이터스케일링의 종류는 점수에 영향을 미치지 않는다고 봐도 될듯,
MinMaxScaler가 큰차이가 없다는건 이상치가 없다는 얘기인듯. 그러니 RobustScaler도 큰 차이가 없는듯?

특성공학 추가

In [ ]:
import pandas as pd
bike=pd.read_csv('bike.csv')
bike['datetime']=pd.to_datetime(bike['datetime'])
bike["year"] = bike["datetime"].dt.year
bike["month"] = bike["datetime"].dt.month
bike["day"] = bike["datetime"].dt.day
bike["hour"] = bike["datetime"].dt.hour
target=bike['count']
data=bike.drop(['datetime','casual','registered','count'],axis=1)

# 특성공학
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
poly=PolynomialFeatures(include_bias=False,degree=2)
data_poly=poly.fit_transform(data)

from sklearn.model_selection import train_test_split
train_data, test_data, train_target, test_target = train_test_split(data_poly, target,random_state=10)

ss=StandardScaler()
train_scaled=ss.fit_transform(train_data)
test_scaled=ss.transform(test_data)

from sklearn.linear_model import LinearRegression
lr=LinearRegression()
lr.fit(train_scaled, train_target)
score=lr.score(test_scaled,test_target)

from sklearn.model_selection import cross_val_score
cross_val=cross_val_score(lr,test_scaled, test_target,cv=5)

print(score)
print(cross_val)
print(cross_val.mean())

0.543430680628272
[0.53712992 0.53457535 0.55705734 0.53317808 0.53060351]
0.538508841059507


랜덤포레스트 회귀 모델

In [ ]:
import pandas as pd
bike=pd.read_csv('bike.csv')
bike['datetime']=pd.to_datetime(bike['datetime'])
bike["year"] = bike["datetime"].dt.year
bike["month"] = bike["datetime"].dt.month
bike["day"] = bike["datetime"].dt.day
bike["hour"] = bike["datetime"].dt.hour
target=bike['count']
data=bike.drop(['datetime','casual','registered','count'],axis=1)

from sklearn.model_selection import train_test_split
train_data, test_data, train_target, test_target=train_test_split(data,target,test_size=0.1, random_state=10)

# 랜덤포레스트 회귀 모델
from sklearn.ensemble import RandomForestRegressor
rf=RandomForestRegressor()
rf.fit(train_data,train_target)
score=rf.score(test_data,test_target)

from sklearn.model_selection import cross_val_score
cross_val=cross_val_score(rf, test_data, test_target,cv=5)

print(score)
print(cross_val)
print(cross_val.mean())

0.9448578322356016
[0.85673643 0.82522269 0.82271356 0.73516254 0.89435216]
0.8268374785511565


위 아래의 랜덤포레스트회귀 모델은 random_state가 동일해도 score가 다르게 나오는데 이유 확인하기
- 트리가 다르게 구성되는거 같은데 random_state로 seed값을 주듯이 트리기반모델에도 seed 값이 있는지 확인하기
- 트리기반모델도 random_state 값을 줘서 시드 고정이 가능하다고 함

In [ ]:
import pandas as pd
bike=pd.read_csv('bike.csv')
bike['datetime']=pd.to_datetime(bike['datetime'])
bike["year"] = bike["datetime"].dt.year
bike["month"] = bike["datetime"].dt.month
bike["day"] = bike["datetime"].dt.day
bike["hour"] = bike["datetime"].dt.hour
target=bike['count']
data=bike.drop(['datetime','casual','registered','count'],axis=1)

from sklearn.model_selection import train_test_split
train_data, test_data, train_target, test_target=train_test_split(data,target,test_size=0.1, random_state=10)

# 랜덤포레스트 회귀 모델
from sklearn.ensemble import RandomForestRegressor
rf=RandomForestRegressor()
rf.fit(train_data,train_target)
score=rf.score(test_data,test_target)

from sklearn.model_selection import cross_val_score
cross_val=cross_val_score(rf, test_data, test_target,cv=5)

print(score)
print(cross_val)
print(cross_val.mean())

0.9461290700759892
[0.85713732 0.8308775  0.81175547 0.72510656 0.88895776]
0.8227669220627394


In [ ]:
import pandas as pd
bike=pd.read_csv('bike.csv')
bike['datetime']=pd.to_datetime(bike['datetime'])
bike["year"] = bike["datetime"].dt.year
bike["month"] = bike["datetime"].dt.month
bike["day"] = bike["datetime"].dt.day
bike["hour"] = bike["datetime"].dt.hour
target=bike['count']
data=bike.drop(['datetime','casual','registered','count'],axis=1)

# 특성공학 추가
from sklearn.preprocessing import PolynomialFeatures
pf=PolynomialFeatures(include_bias=True, degree=2)
data_poly=pf.fit_transform(data)

from sklearn.model_selection import train_test_split
train_data, test_data, train_target, test_target=train_test_split(data_poly,target,test_size=0.1,random_state=10)

from sklearn.ensemble import RandomForestRegressor
rf=RandomForestRegressor()
rf.fit(train_data,train_target)
score=rf.score(test_data,test_target)

from sklearn.model_selection import cross_val_score
cross_val=cross_val_score(rf, test_data, test_target,cv=5)

print(score)
print(cross_val)
print(cross_val.mean())

0.9434691920647639
[0.86462025 0.82500664 0.83255241 0.75899781 0.89204197]
0.8346438168139552


특성공학으로 제곱을 넣어도 큰 차이 없어서 특성공학 삭제하고 시뮬레이션 추가

In [ ]:
import pandas as pd
bike=pd.read_csv('bike.csv')
bike['datetime']=pd.to_datetime(bike['datetime'])
bike["year"] = bike["datetime"].dt.year
bike["month"] = bike["datetime"].dt.month
bike["day"] = bike["datetime"].dt.day
bike["hour"] = bike["datetime"].dt.hour
target=bike['count']
data=bike.drop(['datetime','casual','registered','count'],axis=1)

from sklearn.model_selection import train_test_split
train_data, test_data, train_target, test_target=train_test_split(data,target,test_size=0.1,random_state=10)

from sklearn.ensemble import RandomForestRegressor
rf=RandomForestRegressor()
rf.fit(train_data,train_target)
score=rf.score(test_data,test_target)

# 시뮬레이션 추가
from sklearn.model_selection import GridSearchCV
dic={'max_depth' : range(5,16)}
gs=GridSearchCV(estimator=rf, param_grid=dic, cv=5)
gs.fit(train_data, train_target)
best_para=gs.best_params_
gs_result=gs.cv_results_
best_est=gs.best_estimator_

print(score)
print(best_para)
print(best_est)
best_est.score(test_data, test_target)

0.9459596371657145
{'max_depth': 15}
RandomForestRegressor(max_depth=15)


0.9441877263039715

best_params_를 직접 넣어도 score가 다른데 이유 확인하기 같은 random_state를 넣어도 값이 조금씪 차이있는거랑 같은 이유인거 같음

In [ ]:
import pandas as pd
bike=pd.read_csv('bike.csv')
bike['datetime']=pd.to_datetime(bike['datetime'])
bike["year"] = bike["datetime"].dt.year
bike["month"] = bike["datetime"].dt.month
bike["day"] = bike["datetime"].dt.day
bike["hour"] = bike["datetime"].dt.hour
target=bike['count']
data=bike.drop(['datetime','casual','registered','count'],axis=1)

from sklearn.model_selection import train_test_split
train_data, test_data, train_target, test_target=train_test_split(data,target,test_size=0.1, random_state=10)

# 랜덤포레스트 회귀 모델
from sklearn.ensemble import RandomForestRegressor
rf=RandomForestRegressor(max_depth=15)
rf.fit(train_data,train_target)
score=rf.score(test_data,test_target)

from sklearn.model_selection import cross_val_score
cross_val=cross_val_score(rf, test_data, test_target,cv=5)

print(score)
print(cross_val)
print(cross_val.mean())

0.9447767082602284
[0.85521985 0.83272579 0.8272398  0.73981327 0.89422817]
0.8298453766272944


-----여기 아래는 검색해서 알게된 정보 사용-----

season, holiday, workingday, year, month, day, hour는 말그대로 구분하기 위한 숫자값임
- one hot encoding 사용

In [ ]:
import pandas as pd
bike=pd.read_csv('bike.csv')
bike['datetime']=pd.to_datetime(bike['datetime'])
bike["year"] = bike["datetime"].dt.year
bike["month"] = bike["datetime"].dt.month
bike["day"] = bike["datetime"].dt.day
bike["hour"] = bike["datetime"].dt.hour
target=bike['count']
data=bike.drop(['datetime','casual','registered','count'],axis=1)

In [ ]:
data2=pd.get_dummies(data,columns=['year','month','day','hour','holiday','workingday','season','weather'])
data2.columns

Index(['temp', 'atemp', 'humidity', 'windspeed', 'year_2011', 'year_2012',
       'month_1', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6',
       'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 'month_12',
       'day_1', 'day_2', 'day_3', 'day_4', 'day_5', 'day_6', 'day_7', 'day_8',
       'day_9', 'day_10', 'day_11', 'day_12', 'day_13', 'day_14', 'day_15',
       'day_16', 'day_17', 'day_18', 'day_19', 'hour_0', 'hour_1', 'hour_2',
       'hour_3', 'hour_4', 'hour_5', 'hour_6', 'hour_7', 'hour_8', 'hour_9',
       'hour_10', 'hour_11', 'hour_12', 'hour_13', 'hour_14', 'hour_15',
       'hour_16', 'hour_17', 'hour_18', 'hour_19', 'hour_20', 'hour_21',
       'hour_22', 'hour_23', 'holiday_0', 'holiday_1', 'workingday_0',
       'workingday_1', 'season_1', 'season_2', 'season_3', 'season_4',
       'weather_1', 'weather_2', 'weather_3', 'weather_4'],
      dtype='object')

one hot encoding을 선형회귀모델에 적용하기

In [ ]:
import pandas as pd
bike=pd.read_csv('bike.csv')
bike['datetime']=pd.to_datetime(bike['datetime'])
bike["year"] = bike["datetime"].dt.year
bike["month"] = bike["datetime"].dt.month
bike["day"] = bike["datetime"].dt.day
bike["hour"] = bike["datetime"].dt.hour
target=bike['count']
data=bike.drop(['datetime','casual','registered','count'],axis=1)

# one hot encoding
data2=pd.get_dummies(data,columns=['year','month','day','hour','holiday','workingday','season','weather'])

from sklearn.model_selection import train_test_split
train_data, test_data, train_target, test_target = train_test_split(data2, target,random_state=10)

from sklearn.preprocessing import StandardScaler
ss=StandardScaler()
train_scaled=ss.fit_transform(train_data)
test_scaled=ss.transform(test_data)

from sklearn.linear_model import LinearRegression
lr=LinearRegression()
lr.fit(train_scaled, train_target)
score=lr.score(test_scaled,test_target)

from sklearn.model_selection import cross_val_score
cross_val=cross_val_score(lr,test_scaled, test_target,cv=5)

print('단독점수:',score)
print('교차검증:', cross_val)
print('교차검증 평균:',cross_val.mean())

# 값이 개선은 됨

단독점수: 0.6909127521770053
교차검증: [0.71324969 0.66627752 0.68268612 0.67790655 0.67530028]
교차검증 평균: 0.6830840322421762


In [ ]:
import pandas as pd
bike=pd.read_csv('bike.csv')
bike['datetime']=pd.to_datetime(bike['datetime'])
bike["year"] = bike["datetime"].dt.year
bike["month"] = bike["datetime"].dt.month
bike["day"] = bike["datetime"].dt.day
bike["hour"] = bike["datetime"].dt.hour
target=bike['count']
data=bike.drop(['datetime','casual','registered','count'],axis=1)

# one hot encoding
data2=pd.get_dummies(data,columns=['year','month','day','hour','holiday','workingday','season','weather'])

# 특성공학
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
poly=PolynomialFeatures(include_bias=False,degree=2)
data_poly=poly.fit_transform(data2)

from sklearn.model_selection import train_test_split
train_data, test_data, train_target, test_target = train_test_split(data_poly, target,random_state=10)

ss=StandardScaler()
train_scaled=ss.fit_transform(train_data)
test_scaled=ss.transform(test_data)

from sklearn.linear_model import LinearRegression
lr=LinearRegression()
lr.fit(train_scaled, train_target)
score=lr.score(test_scaled,test_target)

from sklearn.model_selection import cross_val_score
cross_val=cross_val_score(lr,test_scaled, test_target,cv=5)

print(score)
print(cross_val)
print(cross_val.mean())

# 특성공학은 하면 안될듯

0.9073625521625163
[-1.07881869e+23 -8.44484514e+22 -2.31378405e+20 -6.41397155e+23
 -4.06951665e+23]
-2.4818210353910164e+23


one hot encoding 랜덤포레스트에 적용
- 적용안한것과 차이가 있을까?

In [ ]:
import pandas as pd
bike=pd.read_csv('bike.csv')
bike['datetime']=pd.to_datetime(bike['datetime'])
bike["year"] = bike["datetime"].dt.year
bike["month"] = bike["datetime"].dt.month
bike["day"] = bike["datetime"].dt.day
bike["hour"] = bike["datetime"].dt.hour
target=bike['count']
data=bike.drop(['datetime','casual','registered','count'],axis=1)
data2=pd.get_dummies(data,columns=['year','month','day','hour','holiday','workingday','season','weather'])

from sklearn.model_selection import train_test_split
train_data, test_data, train_target, test_target=train_test_split(data2,target,test_size=0.1,random_state=10)

from sklearn.ensemble import RandomForestRegressor
rf=RandomForestRegressor()
rf.fit(train_data,train_target)
score=rf.score(test_data,test_target)

from sklearn.model_selection import cross_val_score
cross_val=cross_val_score(rf, test_data, test_target,cv=5)

print(score)
print(cross_val)
print(cross_val.mean())

0.9242051253044485
[0.8246899  0.76835223 0.689725   0.687803   0.79031403]
0.7521768303779853


In [ ]:
import pandas as pd
bike=pd.read_csv('bike.csv')
bike['datetime']=pd.to_datetime(bike['datetime'])
bike["year"] = bike["datetime"].dt.year
bike["month"] = bike["datetime"].dt.month
bike["day"] = bike["datetime"].dt.day
bike["hour"] = bike["datetime"].dt.hour
target=bike['count']
data=bike.drop(['datetime','casual','registered','count'],axis=1)
data2=pd.get_dummies(data,columns=['year','month','day','hour','holiday','workingday','season','weather'])

from sklearn.model_selection import train_test_split
train_data, test_data, train_target, test_target=train_test_split(data2,target,test_size=0.1,random_state=10)

from sklearn.ensemble import RandomForestRegressor
rf=RandomForestRegressor()
rf.fit(train_data,train_target)
score=rf.score(test_data,test_target)

# 시뮬레이션
from sklearn.model_selection import GridSearchCV
dic={'max_depth' : range(5,16)}
gs=GridSearchCV(estimator=rf, param_grid=dic, cv=5)
gs.fit(train_data, train_target)
best_para=gs.best_params_
gs_result=gs.cv_results_
best_est=gs.best_estimator_

print(score)
print(best_para)
print(best_est)
best_est.score(test_data, test_target)

0.9242871261664003
{'max_depth': 15}
RandomForestRegressor(max_depth=15)


0.8632765335613826

선형회귀모델은 one hot encoding을 사용해서 모델성능을 개선시키는데, 랜덤포레스트는 스케일링이 필요없는 만큼 one hot encoding이 필요없다는 것도 알지만 굳이 하면 점수가 떨어지는데 왜 그러는지 확인 필요
- 특성의 가지수가 많아져서 과대적합이 일어나는건지
- 결국 복잡도가 증가해서 생기는 문제 같은데 복잡도에 대해 공부하기

시간이 없어서 해보지는 못햇는데
검색해보니 선형회귀모델은 log함수 사용해서도 한다고 함